In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
import datetime
import sys
import logging

In [ ]:
def init_database():
    if (os.path.exists('../../data/database') is not True):
        os.mkdir('../../data/database')
    files = os.listdir('../../data/database')
    if 'tester_info.csv' in files:
        global all_tester_info
        all_tester_info = pd.DataFrame.from_csv('../../data/database/tester_info.csv')
    if 'sensor_data.csv' in files:
        global all_sensor_df
        all_sensor_df = pd.DataFrame.from_csv('../../data/database/sensor_data.csv')

In [3]:
useful_sensor = [1, 2, 3, 4, 11, 26, 17, 9, 10]
min_samples = 100
all_tester_info = None
all_sensor_df = None
init_database()

In [4]:
def count_fequency(df):
    # only select the useful information
    df = df[df.SENSORTYPE.isin(useful_sensor)]
    min_time = df.TIMESTAMP.min()
    max_time = df.TIMESTAMP.max()
    c = Counter(df.SENSORTYPE)
    print ("Count per 1 second")
    for i in c.keys():
        c[i] = c[i] / (float(max_time - min_time) /1000.0)

    print(c)

In [5]:
def get_new_tester_id():
    if all_tester_info is None:
        return int(0)
    else:
        return int(all_tester_info.index.max() + 1)

In [6]:
def save_user_info_into_database(tag_df):
    global all_tester_info
    
    cur_user_info = pd.DataFrame([tag_df.iloc[0].tolist()[2:]], columns=tag_df.columns[2:].values.tolist(), index=[0])
    cur_user_info['start_time'] = pd.Series([tag_df.TimeStamp.min()])
    cur_user_info['end_time'] = pd.Series([tag_df.TimeStamp.max() + pd.Timedelta('9 seconds')])
    
    if all_tester_info is None:
        cur_user_info.to_csv("../../data/database/tester_info.csv")
        all_tester_info = pd.DataFrame.from_csv('../../data/database/tester_info.csv')
    else:
        all_tester_info = all_tester_info.append(cur_user_info, ignore_index = True)
        all_tester_info.to_csv('../../data/database/tester_info.csv')


In [7]:
def save_sensor_data_into_database(sensor_df, to_csv=True):
    global all_sensor_df
    
    cur_sensor_df = sensor_df[~sensor_df.TagName.isnull()]
    
    if all_sensor_df is None:
        cur_sensor_df.to_csv('../../data/database/sensor_data.csv')
        all_sensor_df = pd.DataFrame.from_csv('../../data/database/sensor_data.csv')
    else:
        all_sensor_df = all_sensor_df.append(cur_sensor_df, ignore_index = True)
        if to_csv is True:
            all_sensor_df.to_csv('../../data/database/sensor_data.csv')
        
    print("saved")

In [8]:
def read_tag_data(dir):
    tag_data = pd.read_csv(dir, skipinitialspace= True)
    tag_data.TimeStamp = pd.DataFrame(index = pd.to_datetime(tag_data.TimeStamp, utc = 'True')).tz_localize('Asia/Singapore').index
    return tag_data

def read_sensor_data(dir):
    with open(dir,'r') as f:
        first_line = f.readline()
        if first_line.startswith("Found sensor 0 "):
            sensor_data = pd.read_csv(dir, skiprows=13, skipinitialspace= True)
        else:
            sensor_data = pd.read_csv(dir, skipinitialspace= True)
    count_fequency(sensor_data)
    sensor_data.TIMESTAMP = pd.DataFrame(index = pd.to_datetime(sensor_data.TIMESTAMP, unit='ms', utc = 'True')).tz_localize('utc').tz_convert('Asia/Singapore').index
    return sensor_data

In [9]:
def time_difference(sensor_data, tag_data):
    sensor_time_calibration = sensor_data[sensor_data.SENSORTYPE == -1]
    tag_time_calibration = tag_data[tag_data.TagName == 'TIME_CALIBRATION']
    if len(sensor_time_calibration) == 0:
        print("No time calibration on Watch! ********")
        return None
    if len(sensor_time_calibration) != len(tag_time_calibration):
        print('The number of times of time calibration on watch and phone is different! ********')
        return None
   
    return np.mean(tag_time_calibration.TimeStamp - sensor_time_calibration.TIMESTAMP)


In [10]:
def check_tag_sequence(tags):
    temp = [i for i in tags if i not in ['ACTION_FINISH', 'TIME_CALIBRATION', 'wear_start', 'wear_stop']]
    tag_sequence = ['1','2','3','4','5','6','7','8','9','0']
    if len(tag_sequence) > len(temp):
        return False
    for i in range(len(tag_sequence)):
        if temp[i] != tag_sequence[i]:
            return False
    return True

In [11]:
# tag_dict={1:'Tag1',"1":"Tag1"}
tag_dict = {}
for i in range(10):
    tag_dict[i] = 'Tag' + str(i)
    tag_dict[str(i)] = 'Tag' + str(i)

In [12]:
def find_last_divide_sign(s):
    res = None
    for i in range(len(s)):
        if s[i] == '/':
            res = i
            
    return res

In [13]:
def move_to_processed_file_folder(directory):
    file_name = directory[find_last_divide_sign(directory)+1:]
    os.rename(directory, "../../data/raw_data/processed_raw_data/" + file_name)

In [14]:
def consecutive_repeated_tag(tags):
    previous_tag = "None"
    previous_index = -99
    res = []
    for i in range(len(tags)):
        if tags[i] in ['ACTION_FINISH', 'TIME_CALIBRATION', 'wear_start', 'wear_stop']:
            continue
        if (previous_tag == tags[i]):
            res.append(previous_index)
        previous_tag = tags[i]
        previous_index = i
    return res

In [15]:
def main(sensordata_dir, tagdata_dir):
# def main():
#     sensor_data = read_sensor_data('../../data/raw_data/SENSORDATA_20171219-174424.txt')
#     tag_data = read_tag_data('../../data/raw_data/Tags_20171219-174712.txt')
    print("Processing file " + sensordata_dir[find_last_divide_sign(sensordata_dir)+1:])
    try:
        sensor_data = read_sensor_data(sensordata_dir)
        tag_data = read_tag_data(tagdata_dir)
    except:
        print("Processing file " + sensordata_dir[find_last_divide_sign(sensordata_dir)+1:] + " ERROR")
        return
    
    time_different_between_wear_phone = time_difference(sensor_data, tag_data)
    if time_different_between_wear_phone is None:
        return
    print("time_different_between_wear_phone " + str(time_different_between_wear_phone))
    sensor_data.TIMESTAMP = sensor_data.TIMESTAMP + time_different_between_wear_phone
        
    user_groups = list(set(tag_data['Tester_Name'].values.tolist()))

    for user in user_groups:
        cur_user_tag_df = tag_data[tag_data['Tester_Name'] == user]
#         if (cur_user_tag_df.iloc[0]['TagName'] != 'wear_start'):
#             logging.error("Cannot find wear_start tag! User: " + str(user))
#             sys.exit()
#         if (len(cur_user_tag_df[cur_user_tag_df['TagName'] == 'wear_start']) > 1):
#             logging.error("There are more than 1 wear_start tags! User: " + str(user))
#             sys.exit()

        cur_user_id = get_new_tester_id()
        tags = cur_user_tag_df.TagName.tolist()
        # remove the consecutive repeated tag
        # only the latest tag will be used
        skip_tag_idx = consecutive_repeated_tag(tags)
        useful_tags_idx = []
        for i in range(len(cur_user_tag_df)):
            if i not in skip_tag_idx:
                useful_tags_idx.append(i)
                
        cur_user_tag_df = cur_user_tag_df.iloc[useful_tags_idx]
        tags = cur_user_tag_df.TagName.tolist()
        
        if (check_tag_sequence(tags) is False):
            print("Tag Sequence Wrong *********")
            return
        for i in range(len(tags)): 
            cur_tag = tags[i]
            if cur_tag in ['ACTION_FINISH', 'TIME_CALIBRATION', 'wear_start', 'wear_stop']:
                continue 
            cur_tag_start_time = (cur_user_tag_df.iloc[i].TimeStamp + pd.Timedelta('3 seconds'))

            if (i == (len(tags) - 1)):
            # if i is the last index
                cur_tag_end_time = (cur_tag_start_time + pd.Timedelta('5 seconds'))
            else:
                cur_tag_end_time = (cur_user_tag_df.iloc[i+1].TimeStamp)
                if (cur_tag_end_time - cur_tag_start_time > pd.Timedelta('5 seconds')):
                    cur_tag_end_time = (cur_tag_start_time + pd.Timedelta('5 seconds'))

            if (len(sensor_data.loc[(sensor_data.TIMESTAMP < cur_tag_end_time) & (sensor_data.TIMESTAMP > cur_tag_start_time)]) < min_samples):
                print("Something wrong with tag " + cur_tag + " for user " + str(user) + " id " + str(cur_user_id) + "**********")
                print("Number of samples is %d Less than threshold %d" % (len(sensor_data.loc[(sensor_data.TIMESTAMP < cur_tag_end_time) & (sensor_data.TIMESTAMP > cur_tag_start_time)]), min_samples))
                
                return

            if cur_tag in tag_dict.keys():
                sensor_data.loc[(sensor_data.TIMESTAMP < cur_tag_end_time) & (sensor_data.TIMESTAMP > cur_tag_start_time), 'TagName'] = tag_dict[cur_tag]
            else:
                sensor_data.loc[(sensor_data.TIMESTAMP < cur_tag_end_time) & (sensor_data.TIMESTAMP > cur_tag_start_time), 'TagName'] = cur_tag
            sensor_data.loc[(sensor_data.TIMESTAMP < cur_tag_end_time) & (sensor_data.TIMESTAMP > cur_tag_start_time), 'tester_id'] = cur_user_id

    save_sensor_data_into_database(sensor_data, to_csv=False)
    save_user_info_into_database(tag_data)
    
    move_to_processed_file_folder(sensordata_dir)
    move_to_processed_file_folder(tagdata_dir)


In [16]:
all_files = os.listdir('../../data/raw_data/')
tag_files = [i for i in all_files if i.startswith('Tags_')]
# count = 0
for tag_file in tag_files:
    datetime = tag_file[5:]
    sensor_file = 'SENSORDATA_' + datetime
    if sensor_file in all_files:
        main('../../data/raw_data/' + sensor_file, '../../data/raw_data/' + tag_file)
#         count += 1
#     if count > 10:
#         break
        
all_sensor_df.to_csv('../../data/database/sensor_data.csv')

Processing file SENSORDATA_20180108-154729.txt
Count per 1 second
Counter({3: 188.32685972036938, 11: 188.32685972036938, 4: 188.06152757611716, 1: 181.7967408368281, 10: 181.7967408368281, 9: 181.7967408368281, 2: 98.69618732449383, 26: 0.007370337340340067})
time_different_between_wear_phone 0 days 00:00:01.100666
saved
Processing file SENSORDATA_20180108-153126.txt
Count per 1 second
Counter({11: 191.0549790782153, 3: 191.0549790782153, 4: 190.8480603283111, 10: 182.1651364897384, 1: 182.1651364897384, 9: 182.1651364897384, 2: 98.93015342642123, 26: 0.007663657403859418})
time_different_between_wear_phone 0 days 00:00:01.122000
saved
Processing file SENSORDATA_20171221-182254.txt
Count per 1 second
Counter({11: 189.65759739955305, 3: 189.65759739955305, 4: 189.391926737408, 1: 182.51574489365362, 2: 98.61069871384144, 26: 0.09376611605119631})
time_different_between_wear_phone -1 days +23:56:13.700000
saved
Processing file SENSORDATA_20180116-153346.txt
Count per 1 second
Counter({1

saved
Processing file SENSORDATA_20180205-163957.txt
Count per 1 second
Counter({3: 187.87634331153302, 11: 187.87634331153302, 4: 187.51620617131002, 10: 181.32184735947448, 1: 181.32184735947448, 9: 181.32184735947448, 2: 98.47589962257626, 26: 0.1728658273070385})
time_different_between_wear_phone 0 days 00:00:00.397000
saved
Processing file SENSORDATA_20180108-152639.txt
Count per 1 second
Counter({11: 188.99942786359824, 3: 188.99942786359824, 4: 188.7439106356269, 1: 182.6559349430919, 10: 182.6559349430919, 9: 182.6559349430919, 2: 98.6240952745977, 26: 0.016664167041610427})
time_different_between_wear_phone 0 days 00:00:01.196000
saved
Processing file SENSORDATA_20180109-091018.txt
Count per 1 second
Counter({11: 189.89818358259183, 3: 189.89818358259183, 4: 189.59296911073025, 9: 182.87042471767663, 10: 182.87042471767663, 1: 182.87042471767663, 2: 98.81122876215967, 26: 0.039130060495073525})
time_different_between_wear_phone 0 days 00:00:00.345666
saved
Processing file SENS

saved
Processing file SENSORDATA_20180127-115816.txt
Count per 1 second
Counter({3: 178.92044560854015, 11: 178.92044560854015, 4: 178.70844645653676, 1: 171.1175090783508, 9: 171.1175090783508, 10: 171.1175090783508, 2: 98.85999165680757, 26: 0.0068386823226900635})
time_different_between_wear_phone -1 days +23:59:59.902333
saved
Processing file SENSORDATA_20180118-125938.txt
Count per 1 second
Counter({11: 189.97561297584224, 3: 189.97561297584224, 4: 189.67685456023455, 9: 181.89524001417365, 1: 181.89524001417365, 10: 181.89524001417365, 2: 98.92377491679926, 26: 0.03473935065205761})
time_different_between_wear_phone 0 days 00:00:01.210666
saved
Processing file SENSORDATA_20180103-174440.txt
Count per 1 second
Counter({11: 190.49013986188328, 3: 190.49013986188328, 4: 190.253297100546, 9: 182.8987060905986, 1: 182.8987060905986, 10: 182.8987060905986, 2: 98.7509660691581, 26: 0.04986163396574506})
time_different_between_wear_phone 0 days 00:00:00.025000
saved
Processing file SENSO

saved
Processing file SENSORDATA_20180108-163049.txt
Count per 1 second
Counter({11: 189.9890988372093, 3: 189.9890988372093, 4: 189.7468507751938, 1: 182.9093992248062, 10: 182.9093992248062, 9: 182.9093992248062, 2: 98.49806201550388, 26: 0.024224806201550386})
time_different_between_wear_phone 0 days 00:00:00.354000
saved
Processing file SENSORDATA_20180103-144428.txt
Count per 1 second
Counter({3: 188.70491248146288, 11: 188.70491248146288, 4: 188.47137402351734, 10: 181.6637279744042, 1: 181.6637279744042, 9: 181.6637279744042, 2: 98.60577540606499, 26: 0.04086923014047338})
time_different_between_wear_phone 0 days 00:00:00.859333
saved
Processing file SENSORDATA_20180103-150059.txt
Count per 1 second
Counter({11: 188.8165435633534, 3: 188.8165435633534, 4: 188.55495893121275, 10: 181.73949024289323, 1: 181.73949024289323, 9: 181.73949024289323, 2: 98.62691848547254, 26: 0.038048673765915045})
time_different_between_wear_phone 0 days 00:00:00.592666
saved
Processing file SENSORDAT

saved
Processing file SENSORDATA_20180109-101012.txt
Count per 1 second
Counter({11: 191.0968115855879, 3: 191.0968115855879, 4: 190.82931468780885, 1: 183.35426248876144, 9: 183.35426248876144, 10: 183.35426248876144, 2: 98.90697795379735, 26: 0.08173516321026})
time_different_between_wear_phone 0 days 00:00:01.127000
saved
Processing file SENSORDATA_20180109-102122.txt
Count per 1 second
Counter({3: 189.13453043672632, 11: 189.13453043672632, 4: 188.80988621486125, 9: 182.19459493980779, 10: 182.19459493980779, 1: 182.19459493980779, 2: 98.83553843042502, 26: 0.037254254968121})
time_different_between_wear_phone 0 days 00:00:00.729333
saved
Processing file SENSORDATA_20180109-100523.txt
Count per 1 second
Counter({3: 189.84509466437177, 11: 189.84509466437177, 4: 189.5582329317269, 1: 182.36119079492573, 10: 182.36119079492573, 9: 182.36119079492573, 2: 98.67406132466373, 26: 0.04462293618920125})
time_different_between_wear_phone 0 days 00:00:00.859000
saved
